### Importing required libraries

In [ ]:
import torch
import torchvision 
from torchvision.datasets import FashionMNIST
from torchvision import datasets, models
import torchvision.transforms as transforms
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, ConcatDataset, random_split
import matplotlib.pyplot as plt
import numpy as np
import glob

### Importing the pretrained VGG Net16 model

In [2]:
vggnet = torchvision.models.vgg16(weights=True)

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [3]:
vggnet

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Loading the iNaturalist Dataset 

In [4]:
def load_dataset(data_augmentation , train_path, test_path, train_batch_size, val_batch_size, test_batch_size):
    transformer1 = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.4602, 0.4495, 0.3800], std=[0.2040, 0.1984, 0.1921])
    ])
    train_Dataset = torchvision.datasets.ImageFolder(train_path, transform=transformer1)
    train_datasize = int(0.8 * len(train_Dataset))
    train_Dataset, val_Dataset = random_split(train_Dataset, [train_datasize, len(train_Dataset) - train_datasize])
    if data_augmentation == True: 
        transformer2 = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(0.5), 
        transforms.RandomVerticalFlip(0.02),
        transforms.RandomRotation(degrees=45),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4602, 0.4495, 0.3800], std=[0.2040, 0.1984, 0.1921])
        ])
        augmented_dataset = torchvision.datasets.ImageFolder(train_path, transform=transformer2)
        augmented_dataset_size = int(0.2 * len(augmented_dataset))
        augmented_dataset, _  =  random_split(augmented_dataset, [augmented_dataset_size, len(augmented_dataset) - augmented_dataset_size])
        train_Dataset = ConcatDataset([train_Dataset, augmented_dataset])
    train_Loader = DataLoader(
        train_Dataset, 
        batch_size = train_batch_size,
        shuffle=True)
    test_Loader = DataLoader(
        test_path,
        batch_size=test_batch_size, 
        shuffle=True)
    val_Loader = DataLoader(
        val_Dataset, 
        batch_size=val_batch_size, 
        shuffle=True)
    return train_Loader, val_Loader, test_Loader


train_path = '/kaggle/input/inaturalist12k/Data/inaturalist_12K/train/'
test_path = '/kaggle/input/inaturalist12k/Data/inaturalist_12K/val/'
train_batch_size = 64
test_batch_size = 16
val_batch_size = 16
is_Data_Augmentation = True
train_Loader, val_Loader, test_Loader = load_dataset(is_Data_Augmentation, train_path, test_path, train_batch_size, val_batch_size, test_batch_size)
print(len(train_Loader), len(val_Loader), len(test_Loader))   

157 125 4


### Setting device to 'cuda' if GPU is available.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Freezing the Model Parameters

In [7]:
for param in vggnet.parameters():
    param.requires_grad = False

### Adding One more layer to the model.

In [8]:
vggnet.classifier[6] = torch.nn.Linear(in_features = 4096, out_features = 10)
vggnet.classifier.add_module("7", torch.nn.LogSoftmax(dim=1))
vggnet

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
vggnet.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Function to train the model which logs Train accuracy, Train loss, Validataion accuracy & Validation loss 

In [10]:
def train(model, learning_rate, epochs, train_Loader, val_Loader, train_count, test_count, is_wandb_log): 
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay = 1e-4)

    for epoch in range(epochs):
        train_accuracy = 0
        train_loss = 0
        model.train()
        for i, (images, labels) in enumerate(train_Loader):

            images, labels = images.to(device), labels.to(device)
            # doing zero gradient.
            optimizer.zero_grad()

            #forward Propagation
            y_pred = model(images)

            # Calculating Loss.
            loss = loss_function(y_pred, labels)

            # Backward Propagation
            loss.backward()

            # update rule
            optimizer.step()

            train_loss += loss.item()

            _, prediction = torch.max(y_pred.data, 1)
            train_accuracy += int(torch.sum(prediction == labels.data))
    
        train_accuracy /= train_count
        train_loss /= train_count
        print(f"Epochs : {epoch+1} Train Accuracy : {train_accuracy} Train Loss {train_loss}")
    
        test_accuracy = 0
        test_loss = 0
        with torch.no_grad():
            model.eval()
            for i, (images, labels) in enumerate(val_Loader):
                images, labels = images.to(device), labels.to(device)

                y_pred = model(images)

                loss = loss_function(y_pred, labels)
                test_loss += loss.item()

                _, predicted = torch.max(y_pred.data, 1)

                test_accuracy += int(torch.sum(predicted == labels.data))

            test_accuracy /= test_count
            test_loss /= test_count

            print(f"Epochs : {epoch+1} Validation Accuracy : {test_accuracy} Validation Loss {test_loss}")
            if(is_wandb_log):
                wandb.log({"train_accuracy": train_accuracy, "train_loss" : train_loss, "val_accuracy": test_accuracy, "val_error": test_loss}) 
            


### Return the count of Training dataset & testing dataset

In [11]:
def get_train_test_count(train_path, test_path):
    train_count = len(glob.glob(train_path+'/**/*.jpg'))
    test_count = len(glob.glob(test_path+'/**/*.jpg'))
    print("Training dataset count : ", train_count)
    print("Validation dataset count", test_count)
    return train_count, test_count

In [12]:
train_count, test_count = get_train_test_count(train_path, test_path)

Training dataset count :  9999
Validation dataset count 2000


## Training the model

In [ ]:
import wandb
learning_rate = 0.001
epochs = 8
is_wandb_log = True
run = wandb.init( project='Deep_Learning_Assignment2')
# run.name = 'ac_' + activation_func + '_nf_' + str(num_filters) + '_ff_'+ str(filter_factor)+'_df_'+str(dropout_factor)
train(vggnet, learning_rate, epochs, train_Loader, val_Loader, train_count, test_count, is_wandb_log)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
